In [1]:
# Imports
import json
import re
import warnings
from pathlib import Path
from tqdm import tqdm

import numpy as np  
import pandas as pd  
import rich
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

warnings.filterwarnings('ignore')

dataset_path = Path('../data/State_of_data_BR_2023_Kaggle - df_survey_2023.csv')
df = pd.read_csv(dataset_path)

In [2]:
# Count values of nan in ('P2_h ', 'Faixa salarial')
rich.print(df["('P2_h ', 'Faixa salarial')"].isna().sum())

540

In [3]:
def load_json(file_path: Path) -> dict:
    with open(file_path, 'r', encoding="utf-8") as file:
        return json.load(file)
    
def save_json(file_path: Path, data: dict) -> None:
    with open(file_path, 'w') as file:
        json.dump(data, file, ensure_ascii=False ,indent=4)

# Load the merged question
merged_json_path = Path("../questions_jsons/completions_good_features.json")
questions_merged_dict = load_json(merged_json_path)

In [4]:
# criando um dicionário com o número da pergunta e qual é a pergunta
column_dict = {}

for col in df.columns:

    match = re.match(r"\('(.+?)', '(.+?)'\)", col)
    if match:
        cod, pergunta = match.groups()
        column_dict[cod.replace(" ", "")] = pergunta
    else:
        print(col)

('P6_b_16 ', 'SQL Server Integration Services (SSIS))


In [5]:
column_dict['P6_b_16']='SQL Server Integration Services (SSIS)'

In [6]:
df_treat = df.copy()
df_treat.columns = list(column_dict.keys())

# Manual Inspection

In [7]:
# Manually fix some columns

# rename wrong column
df_treat.rename(columns={'P7_1': 'P7_a'}, inplace=True)
df_treat.rename(columns={'P8_3': 'P8_c'}, inplace=True)

# Switch two columns that were in the wrong order
df_treat.rename(columns={'P7_a': 'P7_a_1', 'P7_a_1': 'P7_a'}, inplace=True) # P7_a_1 (antiga P7_a), has corrupted data
df_treat.rename(columns={'P8_b': 'P8_b_1', 'P8_b_1': 'P8_b'}, inplace=True) 
df_treat.rename(columns={'P8_c': 'P8_c_1', 'P8_c_1': 'P8_c'}, inplace=True) 
df_treat.rename(columns={'P4_a_1': 'P4_a2'}, inplace=True) # P4_a_1 Is not an alternative for P4_a 

# Remove columns
#df_treat.drop(columns=[''], inplace=True) 

# Create new dummy columns with 0 on all rows
#df_treat[''] = 0

# Add columns to dictionary

In [8]:
column = "P4_a"
df_treat_not_null = df_treat[df_treat[column].notnull()]
df_treat_not_null[column].iloc[0]

# df_treat_not_null[["P7_a","P7_a_1","P7_a_2","P7_a_3","P7_a_4","P7_a_5","P7_a_6","P7_a_7","P7_a_8","P7_a_9","P7_a_10"]]

'*Ciência de Dados/Machine Learning/AI: *Desenha e executa experimentos com o objetivo de responder perguntas do negócio; desenvolve modelos preditivos e algoritmos de Machine Learning com o objetivo de otimizar e automatizar a tomada de decisão.'

In [9]:
# Manual Analysis
columns_to_maintain = ["P1_l", "P1_m", "P2_i", "P2_j", "P3_c",
                    "P4_a", # Mas não a P4_a_1
                "P4_b", "P4_c", "P4_d", "P4_e", "P4_g", "P4_j", "P5_b", "P6_a", "P6_b", "P6_g", "P6_h", "P7_a", "P7_b", "P7_d", "P8_a", "P8_b", "P8_c", "P8_d"
                    ]

column_in_doubt = [
    "P2_b", "P2_d", "P2_e", "P2_f", "P2_g", # Dados de emprego atuais da pessoa (cargo, se é gerente, setor, etc.)
    "P2_o", # Critérios que a pessoa acha importante ao escolher uma vaga de emprego (salário, clima, desenvolvimento, etc.)
    "P2_r", # Modelo de trabalho atual da pessoa (remoto, presencial, híbrido). Estou mais para o não
    "P6_c", "P6_d", "P6_e", "P6_f" # Se a empresa que trabalha tem data lake, data warehouse, etc.
    ]

possible_for_ordering = ["P1_l", "P2_e", "P2_g", "P2_i", "P2_j"] # "P2_r"]

target = "P2_h"

possible_for_ordering = possible_for_ordering + [target]

# For now, use both columns_to_maintain and column_in_doubt
# categorical columns and numerical columns
# colunas categóricas
cat_cols = [col for col in df_treat.columns if df_treat[col].dtype=='O' and col in (columns_to_maintain + column_in_doubt)]

# colunas numéricas
num_cols = [col for col in df_treat.columns if (df_treat[col].dtype=='int64' or df_treat[col].dtype=='float64') and col in (columns_to_maintain + column_in_doubt)]

merged_questions = load_json(Path("../questions_jsons/merged_questions.json"))
# Flatten the merged_questions dictionary to remove part divisions
merged_questions_no_part_division = {
    question['col_id']: question
    for part in merged_questions.values()
    for question in part
}


cat_cols_with_alternatives = [cat_col for cat_col in cat_cols if "possiveis_respostas" in merged_questions_no_part_division[cat_col]]

cat_cols_without_alternatives = sorted((set(cat_cols) - set(cat_cols_with_alternatives)))

rich.print(f"Categorical columns: {sorted(cat_cols)}")
rich.print(f"Categorical columns with alternatives: {sorted(cat_cols_with_alternatives)}")
rich.print(f"Categorical columns without alternatives: {cat_cols_without_alternatives}")
rich.print(f"Numerical columns: {sorted(num_cols)}")
rich.print(f"Possible columns for ordering: {sorted(possible_for_ordering)}")

Categorical columns: ['P1_l', 'P1_m', 'P2_b', 'P2_e', 'P2_f', 'P2_g', 'P2_i', 'P2_j', 'P2_o', 'P2_r', 'P3_c', 
'P4_a', 'P4_b', 'P4_c', 'P4_d', 'P4_e', 'P4_g', 'P4_j', 'P5_b', 'P6_a', 'P6_b', 'P6_e', 'P6_g', 'P6_h', 'P7_a', 
'P8_b', 'P8_c']

Categorical columns with alternatives: ['P2_o', 'P3_c', 'P4_a', 'P4_b', 'P4_c', 'P4_d', 'P4_g', 'P4_j', 'P6_a', 
'P6_b', 'P6_h', 'P7_a', 'P8_b', 'P8_c']

Categorical columns without alternatives: ['P1_l', 'P1_m', 'P2_b', 'P2_e', 'P2_f', 'P2_g', 'P2_i', 'P2_j', 'P2_r', 
'P4_e', 'P5_b', 'P6_e', 'P6_g']

Numerical columns: ['P2_d', 'P6_c', 'P6_d', 'P6_f', 'P7_b', 'P7_d', 'P8_a', 'P8_d']

Possible columns for ordering: ['P1_l', 'P2_e', 'P2_g', 'P2_h', 'P2_i', 'P2_j']

# Treat columns for later encoding

In [10]:
# Wrapper class for alternatives, handles creation and normalization
class AlternativeWrapper:
    def __init__(self, alternative_name, alternative_answer):
        # Store original answer and normalized form
        self.alternative_name = alternative_name
        self.alternative_answer = alternative_answer
        self.alternative_answer_normalized = self._normalize_answer(alternative_answer)
        self.alternative_name_from_list = alternative_name + "(FromList)"

    def _normalize_answer(self, answer) -> str:
        # Normalizes the answer to lowercase and removes special characters
        return re.sub(r"[^0-9a-záàâãéèêíïóôõöúçñ\n +]", " ", answer.lower()).rstrip()

    def create_column_for_alternative(self, df):
        # Adds a column for each alternative with a default value of 0
        df[self.alternative_name_from_list] = 0

    def set_column_from_list_to_1(self, df, index):
        # Set the column value to 1 if the alternative is selected
        df.at[index, self.alternative_name_from_list] = 1

def deal_with_special_cases(alternatives: list[AlternativeWrapper]):
    """
    Deals with special cases where the alternatives have some error on them.
    """
    # On P8_b_8, the alternative uses the word "desenvolvo", but the final responses collected uses "utilizo"
    for alternative in alternatives:
        if alternative.alternative_name == "P8_b_8":
            alternative.alternative_answer_normalized = alternative.alternative_answer_normalized.replace("desenvolvo", "utilizo")
    return alternatives

def create_alternative_wrappers_and_columns(alternatives: dict, df: pd.DataFrame) -> list:
    """
    Creates a list of AlternativeWrapper objects and adds corresponding columns to the DataFrame.
    """
    alternative_wrappers = []
    for alternative_name, alternative_answer in alternatives.items():
        wrapper = AlternativeWrapper(alternative_name, alternative_answer)
        wrapper.create_column_for_alternative(df)
        alternative_wrappers.append(wrapper)
    return alternative_wrappers

def normalize_string(input_str: str) -> str:
    """
    Normalizes a string by removing special characters and converting to lowercase.
    """
    return re.sub(r"[^0-9a-záàâãéèêíïóôõöúçñ\n+]", " ", input_str.lower())

def process_row_alternatives(df: pd.DataFrame, index: int, row: pd.Series, col: str, alternatives: list, debug: bool = False):
    """
    Processes a single row to find matching alternatives and update the corresponding columns.
    """
    if pd.notna(row[col]):
        alternatives = deal_with_special_cases(alternatives)
        
        # Normalize the row answer and find alternatives in it
        normalized_answer_str = normalize_string(row[col])
        normalized_alternatives = [alt.alternative_answer_normalized for alt in alternatives]

        
        # Find matches between normalized answers and alternatives
        matched_answers = [alt for alt in normalized_alternatives if alt.replace(" ", "") in normalized_answer_str.replace(" ", "")]
        df.at[index, col + "(List)"] = matched_answers

        # Set the corresponding alternative columns to 1 if matched
        for matched_answer in matched_answers:
            for alternative in alternatives:
                if alternative.alternative_answer_normalized == matched_answer:
                    alternative.set_column_from_list_to_1(df, index)

        handle_unmatched_answers(df, index, col, normalized_answer_str, normalized_alternatives, debug)

def handle_unmatched_answers(df: pd.DataFrame, index: int, col: str, normalized_answers_str: str, normalized_alternatives: str, debug: bool = False):
    """
    Handles situations where there are unmatched answers in the row.
    """
    list_of_answer_str = "  ".join(df.at[index, col + "(List)"])
    normalized_alternatives_str = "  ".join(normalized_alternatives) 
    
    # Check if all answers are found by comparing string lengths
    if len(normalized_answers_str) != len(list_of_answer_str):
        if len(list_of_answer_str) == 0:
            # Likely an "outros" option, set list to empty or consider dropping the row
            df.at[index, col + "(List)"] = []
            # Optionally: df.drop(index, inplace=True)
        if debug:
            if abs(len(normalized_answers_str) - len(list_of_answer_str)) > 200:
                rich.print(f"col {col} at index {index}")
                rich.print(f"Answers found: {df.at[index, col + '(List)']}")
                rich.print(f"Original answers normalized: {normalized_answers_str}")
                rich.print(f"Alternatives normalized: {normalized_alternatives_str}")
                rich.print(len(normalized_answers_str), " | ", len(list_of_answer_str))

def process_dataframe(df: pd.DataFrame, cat_cols_with_alternatives: list, merged_questions_no_part_division: dict, debug: bool = False):
    """
    Main function to process the DataFrame, adding new columns and handling alternative answers.
    """
    for col in tqdm(cat_cols_with_alternatives):
        df[col + "(List)"] = [[] for _ in range(len(df))]

        # Get the possible alternatives for the column and create their wrappers
        alternatives = create_alternative_wrappers_and_columns(
            merged_questions_no_part_division[col]["possiveis_respostas"], df
        )

        # Process each row in the DataFrame for the current column
        for index, row in df.iterrows():
            try:
                process_row_alternatives(df, index, row, col, alternatives, debug)
            except Exception as e:
                rich.print(f"Error on column {col} at index {index}")
                rich.print(f"Value being set: {df.at[index, col + '(List)']}")
                raise e

process_dataframe(df_treat, cat_cols_with_alternatives, merged_questions_no_part_division, True)

  0%|          | 0/14 [00:00<?, ?it/s]

100%|██████████| 14/14 [00:09<00:00,  1.41it/s]


In [11]:
# Save the treated dataframe
df_treat.to_csv("../data/treated_data_regex.csv", index=False)

# Split data into train and test

In [12]:
from sklearn.model_selection import train_test_split

X = df_treat.drop(columns=[target])
y = df_treat[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_concat = pd.concat([X_train, y_train], axis=1)
test_concat = pd.concat([X_test, y_test], axis=1)

In [13]:
# Save concated dataframes
train_concat.to_csv("../data/train_concat_regex.csv", index=False)
test_concat.to_csv("../data/test_concat_regex.csv", index=False)

# Encode Columns

## One Hot Encoding

In [14]:
# Load split datasets 
train_concat = pd.read_csv("../data/train_concat_regex.csv")
test_concat = pd.read_csv("../data/test_concat_regex.csv")

In [15]:
# TRAINING DATA

# Transform the columns that do not have alternatives into one hot encoding

ohe = OneHotEncoder(sparse_output=False, handle_unknown='infrequent_if_exist').set_output(transform="pandas")

ohetransform = ohe.fit_transform(train_concat[cat_cols_without_alternatives])

# Rename all columns to have an (OneHot) identifier
ohetransform.columns = [col + "(OneHot)" for col in ohetransform.columns]

# Merge the one hot encoding columns with the original dataframe
train_concat = pd.concat([train_concat, ohetransform], axis=1)

In [16]:
train_concat[[col for col in train_concat.columns if "(OneHot)" in col]].head(1)

,P1_l_Doutorado ou Phd(OneHot),P1_l_Estudante de Graduação(OneHot),P1_l_Graduação/Bacharelado(OneHot),P1_l_Mestrado(OneHot),P1_l_Não tenho graduação formal(OneHot),P1_l_Prefiro não informar(OneHot),P1_l_Pós-graduação(OneHot),P1_m_Ciências Biológicas/ Farmácia/ Medicina/ Área da Saúde(OneHot),P1_m_Ciências Sociais(OneHot),P1_m_Computação / Engenharia de Software / Sistemas de Informação/ TI(OneHot),...,P6_g_Azure(OneHot),P6_g_Databricks(OneHot),P6_g_Google BigQuery(OneHot),P6_g_IBM(OneHot),P6_g_Oracle(OneHot),P6_g_Postgres/MySQL(OneHot),P6_g_Presto(OneHot),P6_g_Snowflake(OneHot),P6_g_Teradata(OneHot),P6_g_nan(OneHot)
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [17]:
# TEST DATA

# Transform the columns that do not have alternatives into one hot encoding

ohetransform = ohe.transform(test_concat[cat_cols_without_alternatives])

# Rename all columns to have an (OneHot) identifier
ohetransform.columns = [col + "(OneHot)" for col in ohetransform.columns]

# Merge the one hot encoding columns with the original dataframe
test_concat = pd.concat([test_concat, ohetransform], axis=1)

In [18]:
test_concat[[col for col in test_concat.columns if "P2_r" in col]].head(1)

,P2_r,P2_r_Modelo 100% presencial(OneHot),P2_r_Modelo 100% remoto(OneHot),P2_r_Modelo híbrido com dias fixos de trabalho presencial(OneHot),P2_r_Modelo híbrido flexível (o funcionário tem liberdade para escolher quando estar no escritório presencialmente)(OneHot),P2_r_nan(OneHot)
0,Modelo 100% remoto,0.0,1.0,0.0,0.0,0.0


In [19]:
df_treat["P2_f"].unique()

array(['Cientista de Dados/Data Scientist', 'Analista de BI/BI Analyst',
       'Analista de Dados/Data Analyst', nan,
       'Engenheiro de Dados/Arquiteto de Dados/Data Engineer/Data Architect',
       'Analista de Negócios/Business Analyst', 'Outra Opção',
       'Desenvolvedor/ Engenheiro de Software/ Analista de Sistemas',
       'Professor/Pesquisador', 'Analytics Engineer', 'Economista',
       'Estatístico', 'Analista de Suporte/Analista Técnico',
       'Data Product Manager/ Product Manager (PM/APM/DPM/GPM/PO)',
       'Outras Engenharias (não inclui dev)',
       'Analista de Inteligência de Mercado/Market Intelligence',
       'Engenheiro de Machine Learning/ML Engineer/AI Engineer',
       'DBA/Administrador de Banco de Dados'], dtype=object)

## Reduce Memory Usage

In [20]:
train_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4234 entries, 0 to 4233
Columns: 734 entries, P0 to P6_g_nan(OneHot)
dtypes: float64(463), int64(188), object(83)
memory usage: 23.7+ MB


In [21]:
# Before doing the ordinal encoding, reduce the memory usage of the dataframe by transforming every numerical column to a binary column

binary_cols = [col for col in train_concat.columns if train_concat[col].dropna().isin([0, 1, 0.0, 1.0]).all()]
# The len for the binary columns is 652, and the total of float and int types is 653. The missing column is P1_a, which has the age of the person.
print("Len Binary Columns: " + str(len(binary_cols)))

# Convert binary columns to bool type
train_concat[binary_cols] = train_concat[binary_cols].astype(bool)

# For the age of a person, int16 is enough 
train_concat['P1_a'] = train_concat['P1_a'].astype('int16')

train_concat.info()

Len Binary Columns: 650
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4234 entries, 0 to 4233
Columns: 734 entries, P0 to P6_g_nan(OneHot)
dtypes: bool(650), int16(1), object(83)
memory usage: 5.3+ MB


In [22]:
# Before doing the ordinal encoding, reduce the memory usage of the dataframe by transforming every numerical column to a binary column

binary_cols = [col for col in test_concat.columns if test_concat[col].dropna().isin([0, 1, 0.0, 1.0]).all()]
# The len for the binary columns is 652, and the total of float and int types is 653. The missing column is P1_a, which has the age of the person.
print("Len Binary Columns: " + str(len(binary_cols)))

# Convert binary columns to bool type
test_concat[binary_cols] = test_concat[binary_cols].astype(bool)

# For the age of a person, int16 is enough 
test_concat['P1_a'] = test_concat['P1_a'].astype('int16')

test_concat.info()

Len Binary Columns: 650
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1059 entries, 0 to 1058
Columns: 734 entries, P0 to P6_g_nan(OneHot)
dtypes: bool(650), int16(1), object(83)
memory usage: 1.3+ MB


In [23]:
# remove rows with "de R$ 101/mês a R$ 2.000/mês" on P2_h
train_concat = train_concat[train_concat["P2_h"] != "de R$ 101/mês a R$ 2.000/mês"]
test_concat = test_concat[test_concat["P2_h"] != "de R$ 101/mês a R$ 2.000/mês"]

## Ordinal Encoding

In [24]:
# TRAINING

# Transform the columns possible for ordering into ordinal encoding

# Possible columns for ordering: ["P1_l", "P2_e", "P2_g", "P2_i", "P2_j", "P2_r"]

categories = {
    "P1_l": ["Prefiro não informar", "Não tenho graduação formal", "Estudante de Graduação", "Graduação/Bacharelado", "Pós-graduação", "Mestrado", "Doutorado ou Phd"],
    "P2_e": ["Team Leader/Tech Leader", "Supervisor/Coordenador", "Gerente/Head", "Diretor/VP", "Sócio ou C-level (CEO, CDO, CIO, CTO etc)"],
    "P2_g": ["Júnior", "Pleno", "Sênior"],
    "P2_i": ["Não tenho experiência na área de dados", "Menos de 1 ano", "de 1 a 2 anos", "de 3 a 4 anos", "de 4 a 6 anos", "de 5 a 6 anos", "de 7 a 10 anos", "Mais de 10 anos"],
    "P2_j": ["Não tive experiência na área de TI/Engenharia de Software antes de começar a trabalhar na área de dados", "Menos de 1 ano", "de 1 a 2 anos", "de 3 a 4 anos", "de 5 a 6 anos", "de 7 a 10 anos", "Mais de 10 anos"],
    "P2_h": [
        "Menos de R$ 1.000/mês",
        "de R$ 1.001/mês a R$ 2.000/mês",
        "de R$ 2.001/mês a R$ 3.000/mês",
        "de R$ 3.001/mês a R$ 4.000/mês",
        "de R$ 4.001/mês a R$ 6.000/mês",
        "de R$ 6.001/mês a R$ 8.000/mês",
        "de R$ 8.001/mês a R$ 12.000/mês",
        "de R$ 12.001/mês a R$ 16.000/mês",
        "de R$ 16.001/mês a R$ 20.000/mês",
        "de R$ 20.001/mês a R$ 25.000/mês",
        "de R$ 25.001/mês a R$ 30.000/mês",
        "de R$ 30.001/mês a R$ 40.000/mês",
        "Acima de R$ 40.001/mês",
        ]
    # P2_r": ["Modelo 100% remoto", "Modelo híbrido com dias fixos de trabalho presencial", "Modelo 100% presencial"]
}

# Create the ordinal encoder
ordinal_encoder = OrdinalEncoder(categories=[categories[col] for col in possible_for_ordering], dtype=np.int8, unknown_value=-1, handle_unknown="use_encoded_value").set_output(transform="pandas")

# Fit and transform the columns
ordinal_encoder_transform = ordinal_encoder.fit_transform(train_concat[possible_for_ordering])

# Rename all columns to have an (OrdEnc) identifier
ordinal_encoder_transform.columns = [col + "(OrdEnc)" for col in ordinal_encoder_transform.columns]

# Merge the one hot encoding columns with the original dataframe
train_concat = pd.concat([train_concat, ordinal_encoder_transform], axis=1)

In [25]:
train_concat[[col for col in train_concat.columns if "(OrdEnc)" in col]].head(1)

,P1_l(OrdEnc),P2_e(OrdEnc),P2_g(OrdEnc),P2_i(OrdEnc),P2_j(OrdEnc),P2_h(OrdEnc)
0,3,-1,1,2,3,4


In [26]:
# TEST

# Fit and transform the columns
ordinal_encoder_transform = ordinal_encoder.transform(test_concat[possible_for_ordering])

# Rename all columns to have an (OrdEnc) identifier
ordinal_encoder_transform.columns = [col + "(OrdEnc)" for col in ordinal_encoder_transform.columns]

# Merge the one hot encoding columns with the original dataframe
test_concat = pd.concat([test_concat, ordinal_encoder_transform], axis=1)

In [27]:
test_concat[[col for col in test_concat.columns if "(OrdEnc)" in col]].head(1)

,P1_l(OrdEnc),P2_e(OrdEnc),P2_g(OrdEnc),P2_i(OrdEnc),P2_j(OrdEnc),P2_h(OrdEnc)
0,3,-1,0,2,0,4


In [28]:
train_concat["P2_h(OrdEnc)"].unique()

array([ 4,  6,  8,  2,  7,  3,  5,  1, -1, 10,  9, 11, 12,  0], dtype=int8)

# Save Data

In [29]:
# Save the treated dataframe
train_concat.to_csv("../data/train_complete.csv", index=False)
test_concat.to_csv("../data/test_complete.csv", index=False)